In [3]:
import os
import bz2
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
processed_fits_files = set()

def process_entry(row):
    run_padded = f"{row['run']:06d}"
    field_padded = f"{row['field']:04d}"
    bands = ['u', 'g', 'r', 'i', 'z']

    for band in bands:
        fits_filename = f"frame-{band}-{run_padded}-{row['camcol']}-{field_padded}.fits.bz2"
        fits_path = os.path.join(fits_folder, fits_filename)
        decompressed_path = fits_path.replace(".bz2", "")

        # Check if this FITS file has been processed before
        if fits_filename not in processed_fits_files:
            # Decompress the file if it hasn't been processed
            with bz2.open(fits_path, 'rb') as f_in, open(decompressed_path, 'wb') as f_out:
                f_out.write(f_in.read())
            processed_fits_files.add(fits_filename)

        # Load the FITS file
        with fits.open(decompressed_path) as hdulist:
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)

            # Convert RA, Dec to pixel coordinates
            x, y = wcs.all_world2pix(row['ra'], row['dec'], 1)

    # Define the cropping region
            size = 45
            x_min, x_max = int(x - size / 2), int(x + size / 2)
            y_min, y_max = int(y - size / 2), int(y + size / 2)

    # Adjust coordinates to fit within the image boundaries
            y_max = min(y_max, data.shape[0])
            x_max = min(x_max, data.shape[1])
            y_min = max(y_min, 0)
            x_min = max(x_min, 0)

    # Crop the image, pad if necessary
            cropped_data = data[y_min:y_max, x_min:x_max]

    # Check if padding is needed
            pad_y = size - cropped_data.shape[0]
            pad_x = size - cropped_data.shape[1]

            if pad_y > 0 or pad_x > 0:
        # Pad the image with zeros (black) where necessary
                cropped_data = np.pad(cropped_data, ((0, pad_y), (0, pad_x)), 'constant')

    # Save to a new FITS file with objid in the filename to differentiate between crops from the same image
            cropped_filename = f"cropped-{band}-{run_padded}-{row['camcol']}-{field_padded}-{row['objid']}.fits"
            cropped_path = os.path.join(output_folder, cropped_filename)
            hdu = fits.PrimaryHDU(cropped_data, header=wcs[y_min:y_max, x_min:x_max].to_header())
            hdu.writeto(cropped_path, overwrite=True)   

# Paths
csv_path = "/Users/agastya/Desktop/CV/sector10/galaxys_s10_200.csv"
fits_folder = "/Users/agastya/Desktop/CV/sector10/galaxy_FITS"
output_folder = "Cropped_Galaxy_Fits"

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Read the CSV
df = pd.read_csv(csv_path, skiprows=1)  # Skips the first row

# Process each entry
df.apply(process_entry, axis=1)


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa

0      None
1      None
2      None
3      None
4      None
       ... 
195    None
196    None
197    None
198    None
199    None
Length: 200, dtype: object

## Converting to Png

In [4]:
import numpy as np

In [5]:
import os
import imageio
from astropy.io import fits
import numpy as np

def fits_to_png(fits_folder, output_folder, expected_size=(45, 45)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(fits_folder):
        for file in files:
            if file.endswith('.fits'):
                path_to_fits = os.path.join(root, file)
                with fits.open(path_to_fits) as hdul:
                    image_data = hdul[0].data

                    # Check if the image is of the expected size
                    if image_data.shape != expected_size:
                        print(f"Skipping {file}: unexpected size {image_data.shape}")
                        continue

                    # Clip the image to avoid extreme values
                    clipped_image = np.clip(image_data, np.percentile(image_data, 5), np.percentile(image_data, 95))
                    # Normalize the image to the 0-255 range
                    normalized_image = ((clipped_image - clipped_image.min()) * (1/(clipped_image.max() - clipped_image.min()) * 255)).astype('uint8')
                    # Save the image as PNG
                    imageio.imsave(os.path.join(output_folder, file.replace('.fits', '.png')), normalized_image)
fits_to_png('/Users/agastya/Desktop/CV/sector10/Cropped_Galaxy_Fits', '/Users/agastya/Desktop/CV/sector10/Galaxy_PNGs')

In [6]:
import os
import numpy as np
from skimage import io
from natsort import natsorted

def stack_images_by_objid(png_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Dictionary to hold the images for each objid
    images_dict = {}
    
    # Read the PNG files and group them by objid
    for file in natsorted(os.listdir(png_folder)):
        if file.endswith('.png'):
            objid = file.split('-')[-1].split('.')[0]  # Extract the objid from the filename
            band = file.split('-')[1]  # Extract the band from the filename
            if objid not in images_dict:
                images_dict[objid] = {}
            images_dict[objid][band] = os.path.join(png_folder, file)
    
    # For each objid, stack the images for all bands
    for objid, band_paths in images_dict.items():
        # Ensure we have all bands
        if len(band_paths) == 5 and all(band in band_paths for band in ['u', 'g', 'r', 'i', 'z']):
            stacked_image = []
            for band in ['u', 'g', 'r', 'i', 'z']:
                image_path = band_paths[band]
                image = io.imread(image_path, as_gray=True)
                stacked_image.append(image)
            
            stacked_image = np.stack(stacked_image, axis=-1)  # Stack along the third dimension
            np.save(os.path.join(output_folder, f'stacked_{objid}.npy'), stacked_image)
        else:
            print(f"Missing bands for objid {objid}, skipping...")
            
stack_images_by_objid('/Users/agastya/Desktop/CV/sector10/Galaxy_PNGs', '/Users/agastya/Desktop/CV/sector10/Stacked_Galaxy_Images')

In [7]:
import numpy as np
import os

def normalize_stacked_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in os.listdir(input_folder):
        if file.endswith('.npy'):
            # Load the stacked image
            image_path = os.path.join(input_folder, file)
            stacked_image = np.load(image_path)
            
            # Normalize the image data to 0-1
            normalized_image = stacked_image / 255.0
            
            # Save the normalized image
            output_path = os.path.join(output_folder, file)
            np.save(output_path, normalized_image)
            print(f"Normalized and saved {file}")

# Provide the correct path to your input and output folders
input_folder = '/Users/agastya/Desktop/CV/sector10/Stacked_Galaxy_Images'
output_folder = '/Users/agastya/Desktop/CV/sector10/Normalized_Galaxy_Images'
normalize_stacked_images(input_folder, output_folder)

Normalized and saved stacked_1237651249896293137.npy
Normalized and saved stacked_1237651249896096034.npy
Normalized and saved stacked_1237651249895899565.npy
Normalized and saved stacked_1237651249895899808.npy
Normalized and saved stacked_1237651249896423530.npy
Normalized and saved stacked_1237651249895703452.npy
Normalized and saved stacked_1237651249896293095.npy
Normalized and saved stacked_1237651249895440505.npy
Normalized and saved stacked_1237651249895833604.npy
Normalized and saved stacked_1237651249896227068.npy
Normalized and saved stacked_1237651249895768184.npy
Normalized and saved stacked_1237651249895243848.npy
Normalized and saved stacked_1237651249895374955.npy
Normalized and saved stacked_1237651249895374969.npy
Normalized and saved stacked_1237651249895637133.npy
Normalized and saved stacked_1237651249896095933.npy
Normalized and saved stacked_1237651249895243902.npy
Normalized and saved stacked_1237651249896554793.npy
Normalized and saved stacked_12376512498960966

In [8]:
import numpy as np
import os
from scipy.ndimage import rotate

def augment_images(input_folder, output_folder, num_augmented_images_per_file):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in os.listdir(input_folder):
        if file.endswith('.npy'):
            image_path = os.path.join(input_folder, file)
            image = np.load(image_path)
            
            # Generate augmented images
            for i in range(num_augmented_images_per_file):
                # Start with the original image each time
                augmented_image = np.copy(image)
                
                # Randomly decide which augmentation to apply
                if np.random.rand() > 0.5:
                    # Random rotation
                    angle = np.random.uniform(0, 360)
                    augmented_image = rotate(augmented_image, angle, reshape=False, mode='nearest')
                
                # Random horizontal flip
                if np.random.rand() > 0.5:
                    augmented_image = np.fliplr(augmented_image)
                    
                # Random vertical flip
                if np.random.rand() > 0.5:
                    augmented_image = np.flipud(augmented_image)
                
                # Save the augmented image
                augmented_image_path = os.path.join(output_folder, f'aug_{i}_{file}')
                np.save(augmented_image_path, augmented_image)
                print(f"Saved augmented image to {augmented_image_path}")

# Provide the correct path to your input and output folders
input_folder = '/Users/agastya/Desktop/CV/sector10/Normalized_Galaxy_Images'
output_folder = '/Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images'
num_augmented_images_per_file = 25  # How many augmented images you want to generate per original file
augment_images(input_folder, output_folder, num_augmented_images_per_file)

Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_0_stacked_1237651249896293137.npy
Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_1_stacked_1237651249896293137.npy
Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_2_stacked_1237651249896293137.npy
Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_3_stacked_1237651249896293137.npy
Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_4_stacked_1237651249896293137.npy
Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_5_stacked_1237651249896293137.npy
Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_6_stacked_1237651249896293137.npy
Saved augmented image to /Users/agastya/Desktop/CV/sector10/Augmented_Galaxy_Images/aug_7_stacked_1237651249896293137.npy
Saved augmented image to

Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_0_stacked_1237651249896555204.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_1_stacked_1237651249896555204.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_2_stacked_1237651249896555204.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_3_stacked_1237651249896555204.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_4_stacked_1237651249896555204.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_0_stacked_1237651249895309516.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_1_stacked_1237651249895309516.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_2_stacked_12376512

Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_2_stacked_1237651249895703601.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_3_stacked_1237651249895703601.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_4_stacked_1237651249895703601.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_0_stacked_1237651249896030517.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_1_stacked_1237651249896030517.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_2_stacked_1237651249896030517.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_3_stacked_1237651249896030517.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_4_stacked_12376512

Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_0_stacked_1237651249895178514.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_1_stacked_1237651249895178514.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_2_stacked_1237651249895178514.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_3_stacked_1237651249895178514.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_4_stacked_1237651249895178514.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_0_stacked_1237651249895833814.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_1_stacked_1237651249895833814.npy
Saved augmented image to /Users/agastya/Desktop/Work/CV/sector10/Augmented_Galaxy_Images/aug_2_stacked_12376512